# Overview: Flow of Snowpark ML API / MLOps Hands-on-Lab

<img src="assets/End-to-end_demo.png" width="800" />




## Background Information

Tasty Bytes is one of the largest food truck networks in the world with localized menu options spread across 30 major cities in 15 countries. **Tasty Bytes is aiming to achieve 25% YoY sales growth over 5 years.**

As Tasty Bytes Data Scientists, we have been asked to support this goal by helping our food truck drivers more intelligently pick where to park for shifts. 

**We want to direct our trucks to locations that are expected to have the highest sales on a given shift.
This will maximize our daily revenue across our fleet of trucks.**

To provide this insight, we will use historical shift sales at each location to build a model. This data has been made available to us in Snowflake.Our model will provide the predicted sales at each location for the upcoming shift.



<img src="assets/problem_overview.png" width="800" />




<img src="assets/snowpark_101.png" width="800" />




## Import Packages

Just like the Python packages we are importing, we will import the Snowpark modules that we need.
**Value**: Snowflake modules provide efficient ways to work with data and functions in Snowflake.



In [ ]:
# Import Packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
import json
import sys
import cachetools
import getpass

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

#### Need to update snowflake-ml-python package to version 1.2.2 to reflect PuPr Model Registry syntax



## Connect to Snowflake

Our assigned HOL Snowflake role, `<ROLEXXX>`, can access the data in your assigned `<SCHEMAXXX>` of the HOL database. We will use your assigned `<WHXXX>` warehouse that has been created as a dedicated compute for data science workloads.We will use these parameters and our Snowflake account credentials to connect to Snowflake and create a Snowpark session. 

**Value:** Secure and governed.



## Create your Snowflake data connection called "HOL`xxx`"

If you were assigned user`xxx`,

- set Name as HOL,`xxx`
- set Account as XXX-XXX,
- set Warehouse WH,`xxx`,,
- set Database as HOL,
- set Schemas as SCHEMA,`xxx`,,
- set Username as USER,`xxx`,,
- set Password as ,_(this is case sensitive)_
- set User Role as HOL,`xxx`

For instance, if you were assigned number 1, your data connection is:{ "username": "user1", "password": "test", "account": "XXXX-XXXX", "database": "HOL", "role": "HOL1", "warehouse": "HOL1_WH", "schema": "SCHEMA1" }



For instance, if you were assigned number 6, your data connection is:

If you were assigned user6, 

- set Name as HOL6
- set Account as XXX-XXX 
- set Warehouse as WH6
- set Database as HOL6
- set Schemas as ANALYTICS
- use Username & Password for Type
- set Username as USER6
- set Password as XXX ,_(this is case sensitive)_
- set User Role as ROLE6
- check Snowpark integration
- check Allow use in writeback cells
- select Create connection



In [ ]:
# Get account credentials from a json file
with open("hol_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]
    database = data["database"]
    schema = data["schema"]
    role = data["role"]
    warehouse = data["warehouse"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": role,
    "warehouse": warehouse,
    "database": database,
    "schema": schema,
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [ ]:
# Checking environment setting and versions associated with Snowpark connection
from snowflake.snowpark.version import VERSION
print("----------------------------------------")
snowflake_environment = session.sql('select current_warehouse(), current_database(), current_schema(), current_version()').collect()
snowpark_version = VERSION
print('Warehouse                   : {}'.format(snowflake_environment[0][0]))
print('Database                    : {}'.format(snowflake_environment[0][1]))
print('Schema                      : {}'.format(snowflake_environment[0][2]))
print('Snowflake version           : {}'.format(snowflake_environment[0][3]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))
print("----------------------------------------")

# Part 1 - Use Snowpark to access and prepare data for modeling

<img src="assets/Part1.png" width="800" />




## Snowpark DataFrame

Let's create a Snowpark DataFrame containing our shift sales data from the **shift_sales_v** view in our Snowflake account using the Snowpark session.table function. A DataFrame is a data structure that contains rows and columns, similar to a SQL table.

**Value:** Familiar representation of data for Python users.



In [ ]:
snowpark_df = session.table("HOL." + schema +".SHIFT_SALES_V")

## Preview the Data

With our Snowpark DataFrame defined, let’s use the .show() function to take a look at the first 10 rows.
**Value:** Instant access to data.



In [ ]:
# Preview the data
# Use the .show() function to look at the first 10 rows.
# VALUE: instant access to data
snowpark_df.show()

## Select, Filter, Sort

Notice the Null values for "shift_sales". Let's look at a single location.To do this, we will make another Snowpark DataFrame, location_df, from the above DataFrame and we will:

1. Select columns
2. Filter to a single location ID
3. Sort by date

**Value**: Efficient transformation pipelines using Python syntax and chained logic.



In [ ]:
# Select
location_df = snowpark_df.select("date", "shift", "shift_sales", "location_id", "city")

# Filter
location_df = location_df.filter(F.col("location_id") == 1135)

# Sort
location_df = location_df.order_by(["date", "shift"], ascending=[0, 0])

# Display
location_df.show(n=20)

We can see that shift sales are populated 8 days prior to the latest date in the data. The **missing values** represent future dates that do not have shift sales yet.

## Snowpark works in two main ways:

1. Snowpark code translated and executed as SQL on Snowflake
2. Python functions deployed in a secure sandbox in Snowflake



<img src="assets/snowparkoverview.png" width="800" />


Here's the value of using Snowpark:

<img src="assets/SnowparkValue.png" width="800" />

## Explain the Query

Let's look at what was executed in Snowflake to create our location_df DataFrame.

The translated SQL query can be seen in the Snowsight interface under _Activity_ in the _Query History_ or directly in our notebook by using the explain() function. **Value:** Transparent execution and compute usage.



<img src="assets/query_history.png" width="800" />




In [ ]:
location_df.explain()

<img src="assets/data_exploration.png" width="800" />

## Compare DataFrame Size

Let's bring a sample of our Snowflake dataset to our Python environment in a pandas DataFrame using the to_pandas() function. We will compare how much memory is used for the pandas DataFrame compared to the Snowpark DataFrame. As we will see, no memory is used for the Snowpark DataFrame in our Python environment. All data in the Snowpark DataFrame remains on Snowflake.
**Value:** No copies or movement of data when working with Snowpark DataFrames.



In [ ]:
# Bring 10,000 rows from Snowflake to pandas
pandas_df = snowpark_df.limit(10000).to_pandas()

# Get Snowpark DataFrame size
snowpark_size = sys.getsizeof(snowpark_df) / (1024*1024)
print(f"Snowpark DataFrame Size (snowpark_df): {snowpark_size:.2f} MB")

# Get pandas DataFrame size
pandas_size = sys.getsizeof(pandas_df) / (1024*1024)
print(f"Pandas DataFrame Size (pandas_df): {pandas_size:.2f} MB")

## Data Exploration

Here, we will use Snowpark to explore our data. A common pattern for exploration is to use Snowpark to manipulate our data and then bring an aggregate table to our Python environment for visualization.
**Value:** - Native Snowflake performance and scale for aggregating large datasets. - Easy transfer of aggregate data to the client-side environment for visualization.
As we explore our data, we will highlight what is being done in Snowflake and what we are transferring to our client-side environment (Python notebook environment) for visualization.



## How many rows are in our data?

This will give us an idea of how we might need to approach working with this data. Do we have enough data to build a meaningful model? What compute might be required? Will we need to sample the data?
**What's happening where?:** Rows counted in Snowflake. No data transfer.



In [ ]:
#Use the .count() function
snowpark_df.count()

## Let's calculate some descriptive statistics.

We use the Snowpark describe() function to calculate summary statistics and then bring the aggregate results into a pandas DataFrame to visualize in a formatted table.
**What's happening where?:** Summary statistics calculated in Snowflake. Transfer aggregate summary statistics for client-side visualization.



In [ ]:
#Use the Snowpark DataFrame .describe function. You need to need to visualize from a pandas DataFrame
snowpark_df.describe().to_pandas()

In [ ]:
# What are the numeric columns?
# Define Snowflake numeric types
numeric_types = [T.DecimalType, T.DoubleType, T.FloatType, T.IntegerType, T.LongType]

# Get numeric columns
numeric_columns = [col.name for col in snowpark_df.schema.fields if type(col.datatype) in numeric_types]
numeric_columns

In [ ]:
# What are the categorical columns?
# Define Snowflake categorical types
categorical_types = [T.StringType]

# Get categorical columns
categorical_columns = [col.name for col in snowpark_df.schema.fields if type(col.datatype) in categorical_types]
categorical_columns

In [ ]:
# What are the average shift sales (USD) by city?
# Group by city and average shift sales
analysis_df = snowpark_df.group_by("city").agg(F.mean("shift_sales").alias("avg_shift_sales"))

# Sort by average shift sales
analysis_df = analysis_df.sort("avg_shift_sales", ascending=True)

# Pull to pandas and plot
analysis_df.to_pandas().plot.barh(x="CITY", y="AVG_SHIFT_SALES")

<img src="assets/feature_engineering.png" width="800" />

## Feature Engineering

Now let's keep revelant columns and transform columns to create features needed for our prediction model.To make some of our features more useful, we will normalize them using standard preprocessing techniques, such as One-Hot Encoding and MinMaxScaling. With SnowparkML, you can use a standard sklearn-style API to execute fully distributed feature engineering preprocessing tasks on Snowflake compute, with zero data movement. Let's fit a scaler and encoder to our data, then use it to transform the data, producing new feature columns.


**Value:** The Snowpark syntax makes pipelines easy to implement and understand. The syntax also allows for easy migration of Spark pipelines to Snowflake.


**All transformations for feature engineering in this notebook will be executed on Snowflake compute.**

Notice what we haven't had to do? No tuning, maintenance, or operational overhead. We just need a role, warehouse, and access to the data.
**Value**: Near-zero maintenance. Focus on the work that brings value.



## Create a Rolling Average Feature

We will use a Snowflake window function to get a **rolling shift average by location** over time. Window functions allow us to aggregate on a "moving" group of rows.

#### **Step 1. Create a Window**

Our window will partition the data by location and shift. It will order rows by date. It will include all rows prior to the current date of the observation it is aggregating for.



In [ ]:
window_by_location_all_days = (
    Window.partition_by("location_id", "shift")
    .order_by("date")
    .rows_between(Window.UNBOUNDED_PRECEDING, Window.CURRENT_ROW - 1)
)

#### **Step 2. Aggregate across the Window**



In [ ]:
snowpark_df = snowpark_df.with_column(
    "avg_location_shift_sales", 
    F.avg("shift_sales").over(window_by_location_all_days)
)

## Impute Missing Values

The rolling average feature we just created is missing if there are no prior shift sales at that location. We will replace those missing values with 0.



In [ ]:
snowpark_df = snowpark_df.fillna(value=0, subset=["avg_location_shift_sales"])

## Leverage Snowpark ML Modeling API to create features

Snowpark ML provides APIs to support each stage of an end-to-end machine learning development and deployment process and includes two key components: [Snowpark ML Modeling](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling) and [Snowpark ML Ops](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-mlops-model-registry).

[Snowpark ML Modeling](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling) supports data preprocessing, feature engineering, and model training in Snowflake using popular machine learning frameworks, such as scikit-learn, xgboost, and lightgbm. This API also includes a preprocessing module that can use compute resources provided by a Snowpark-optimized warehouse to provide scalable data transformations.

Snowpark ML Operations (MLOps), featuring the [Snowpark ML Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-mlops-model-registry), complements the Snowpark ML Development API. The model registry allows secure deployment and management of models in Snowflake, and supports models trained both inside and outside of Snowflake.



Here are a Snowpark Features commonly used for Data Engineering tasks:

<img src="assets/Snowpark4DE.png" width="800" />




Here are Snowpark Features commonly used by Data Scientists for ML efforts:

<img src="assets/Snowpark4DS.png" width="800" />

In [ ]:
session.sql("alter warehouse " + warehouse + " set warehouse_size = LARGE").collect()

In [ ]:
# Import Snowpark ML: Machine Learning Toolkit for Snowflake
import snowflake.ml.modeling.preprocessing as snowmlpp

# Define our scaler and ordinal encoding functions

# Snowpark ML scaler (MinMaxScaler) is used to shrink data within the given range, usually of 0 to 1. 
# It transforms data by scaling features to a given range. It scales the values to a specific value range without changing the shape of the original distribution.
# For the Tasty_Bytes data, use MinMaxScaler to normalize "CITY_POPULATION" into "CITY_POPULATION_NORM" with values between 0 and 1.

def fit_scaler(session, df):
    mm_target_columns = ["CITY_POPULATION"]
    mm_target_cols_out = ["CITY_POPULATION_NORM"]
    snowml_mms = snowmlpp.MinMaxScaler(input_cols=mm_target_columns, output_cols=mm_target_cols_out)
    snowml_mms.fit(df)
    return snowml_mms

# Snowpark ML ordinal encoding (OE) is used to improve model performance by providing more information to the model about categorical variables. 
# It can help to avoid the problem of ordinality, which can occur when a categorical variable has a natural ordering (e.g. “small”, “medium”, “large”).
# For the Tasty_Bytes data, use OE to change "SHIFT" which is currently AM or PM into and integer representation of "SHIFT_OHE" is 1.0 or 0.0. 

def fit_oe(session, df):
    oe_target_cols = ["SHIFT"]
    oe_output_cols = ["SHIFT_OE"]
    snowml_oe = snowmlpp.OrdinalEncoder(input_cols=oe_target_cols, output_cols=oe_output_cols)
    snowml_oe.fit(df)
    return snowml_oe

In [ ]:
# Run Snowpark ML preprocessing functions against our feature data

# For the Tasty_Bytes data, use MinMaxScaler to normalize "CITY_POPULATION" into "CITY_POPULATION_NORM" with values between 0 and 1.
snowml_mms = fit_scaler(session, snowpark_df)
normed_df = snowml_mms.transform(snowpark_df)

# For the Tasty_Bytes data, use OneHotEncoder to change "SHIFT" which is currently AM or PM into and integer representation of "SHIFT_OHE_AM" is 1 or 0 and "SHIFT_OHE_PM" is 1 or 0. 
snowml_oe = fit_oe(session, normed_df)
oe_df = snowml_oe.transform(normed_df)
oe_df.show()

## Filter to Historical Data

Our data includes placeholders for future data with missing shift sales. The future data represents the next 7 days of shifts for all locations. The historical data has shift sales for all locations where a food truck parked during a shift. We will only use historical data when training our model and will filter out the dates where the shift_sales column is missing.



In [ ]:
# Data Science best practice: Always perform data quality on your training set e.g. remove nulls or invalid cells as they are the biggest problem in a training set as they output high false positives

# Specifically for Tasty_Bytes data, dates where "shift_sales" are null values reflect future dates where sales need to be predicted.
# Filter out these future dates so these records will not be used in model training. 
historical_df = oe_df.filter(F.col("shift_sales").is_not_null())

## Persist Transformations

If we want to save the changes we can either save it as a table, meaning the SQL generated by the DataFrame is executed and the result is stored in a table or as a view where the DataFrame SQL will be the definition of the view.
**save_as_table** saves the result in a table, if **mode='overwrite'** then it will also replace the data that is in it.



In [ ]:
# Let's select 
historical_df.write.save_as_table(table_name=schema + '.INPUT_DATA', mode='overwrite')
session.table('INPUT_DATA').show()

# Part 2 - Use Snowflake Cortex ML-Based Function for Time-Series Forecasting

<img src="assets/Part2.png" width="800" />

## Snowflake Cortex ML Functions

Time-Series Forecasting is part of Snowflake Cortex, Snowflake’s intelligent, fully-managed AI and ML service. This feature is part of the Snowflake Cortex ML-based function suite. Forecasting employs a machine learning algorithm to predict future data by using historical time series data.

<img src="assets/CortexML.png" width="800" />


Time series forecasting produces univariate predictions of future data based on historical input data. A common use case is to forecast sales based on seasonality and other factors.The historical data must include:

- A timestamp column.
- A target value column representing some quantity of interest at each timestamp.


The historical data can also include additional columns that might have influenced the target value ([exogenous variables](https://en.wikipedia.org/wiki/Exogenous_and_endogenous_variables)). These can be numbers or text. The nature (categorical or continuous) of each such column is automatically detected.

This historical data is used to train a machine learning model that produces a forecast of the value column at future timestamps. The model is a schema-level object and can be used for multiple forecasts after it is trained.

Forecasting works with either single-series or multi-series data. Multi-series data represents multiple independent threads of events. For example, if you have sales data for multiple stores, each store’s sales can be forecast separately by a single model based on the store identifier.

<img src="assets/MLPF_Forecast.png" width="800" />

To produce forecasts of time series data, use the Snowflake built-in class [FORECAST](https://docs.snowflake.com/en/sql-reference/classes/forecast), and follow these steps:

1. [Create a forecast model object](https://docs.snowflake.com/en/sql-reference/classes/forecast.html#label-class-forecast-create) passing in a reference to the training data.This object will fit (train) a model to the training data that you provide. The model is a schema-level object.
2. Using this forecast model object, call [CREATE SNOWFLAKE.ML.FORECAST](https://docs.snowflake.com/en/sql-reference/classes/forecast.html#label-class-forecast-create) to produce a forecast, passing in information about the future period (that is, the number of time steps and values for any non-timestamp features).The method uses the model to produce a forecast.



#### About the Forecasting Algorithm

The forecasting algorithm is powered by a [gradient boosting machine](https://en.wikipedia.org/wiki/Gradient_boosting) (GBM). Like an [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) model, it uses a differencing transformation to model data with a non-stationary trend and uses auto-regressive lags of the historical target data as model features.

Additionally, the algorithm uses rolling averages of historical target data to help predict trends and automatically produces cyclic calendar features (such as day of week and week of year) from timestamp data.

You can fit models with only historical target and timestamp data, or you may include exogenous data (features) that might have influenced the target value. Exogenous variables can be numerical or categorical and may be NULL (rows containing NULLs for exogenous variables are not dropped).

The algorithm does not rely on one-hot encoding when training on categorical features, so you can use categorical data with many dimensions (high cardinality).

For more details about Snowflake's ML-Powered Forecasting Algorithm and how to use, please see [https://docs.snowflake.com/en/user-guide/ml-powered-forecasting#about-the-forecasting-algorithm](https://docs.snowflake.com/en/user-guide/ml-powered-forecasting#about-the-forecasting-algorithm)



Here's a short video presenting [Snowflake ML Powered Functions: Forecasting, Anomaly Detection, Contribution Explorer](https://www.youtube.com/watch?v=8WgVTf3im7w)

Check out the code at: [https://github.com/Snowflake-Labs/sf-...](https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbUZVS1dBWGhmYlVJUEt4MUsyWlY1VzVNbHVTUXxBQ3Jtc0ttaURLdms3dE4xeHZuVHVtVmlOZmxOQWRiOXA4RjNweWJuRTBSSkJkMnhxVUJseFhNdnBZQkZVbEVnWGdBbnFsODZ6TlZkeV9wSmo1NzhtbUZnOTdfYWE2czRjSnBmRTNkYWZpRm5lckxTay1lTi1Edw&q=https%3A%2F%2Fgithub.com%2FSnowflake-Labs%2Fsf-samples%2Fblob%2Fmain%2Fsamples%2FML%2520Powered%2520Functions%2FJune%25202023%2520MLPF%2520Demos.sql&v=8WgVTf3im7w)





In [ ]:
session.sql('SELECT * FROM SALES_FORECAST_INPUT LIMIT 10;').collect()

In [ ]:
# Create view for lobster sales
session.sql('''CREATE OR REPLACE VIEW lobster_sales AS (SELECT timestamp, total_sold \
            FROM SALES_FORECAST_INPUT \
            WHERE menu_item_name LIKE 'Lobster Mac & Cheese');''')

In [ ]:
# Create Cortex ML forecast called lobstermac_forecast
session.sql('''CREATE OR REPLACE snowflake.ml.forecast lobstermac_forecast \
            (INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'lobster_sales'), \
            TIMESTAMP_COLNAME => 'TIMESTAMP', \
            TARGET_COLNAME => 'TOTAL_SOLD');''')

In [ ]:
# Show models to confirm training has completed: 
forecast = session.sql('SHOW snowflake.ml.forecast;')
forecast.show()

# Part 3 - Use Snowpark to train a model

# <img src="assets/Part3.png" width="800" />

<img src="assets/model_training.png" width="800" />

## Drop Columns

Let's return to the original prepared table, with all cities listed, and drop columns that will not be used in the model.



In [ ]:
prepared_df = historical_df.drop("location_id", "city_population", "shift", "city", "date")
prepared_df.show()

## Build a simple XGBoost Regression Model on Snowflake

We will now use our training data to train a linear regression model on Snowflake.Recall from above, the two main ways that Snowpark works:

1. Snowpark code translated and executed as SQL on Snowflake
2. Python functions deployed in a secure sandbox in Snowflake

We will be leveraging the deployment of Python functions into Snowflake for training and model deployment.



In [ ]:
# Retrieve column names needed in the next code block to populate feature_column_names
prepared_df.columns

In [ ]:
# Let's define relevant features needed for the prediction model.
LABEL_COLUMNS = ["SHIFT_SALES"]
OUTPUT_COLUMNS = ["PRED_SHIFT_SALES"]
FEATURE_COLUMN_NAMES = ["SHIFT_OE", "CITY_POPULATION_NORM", "MONTH", "DAY_OF_WEEK","LATITUDE","LONGITUDE","AVG_LOCATION_SHIFT_SALES"]

input_df = prepared_df.select(*LABEL_COLUMNS, *FEATURE_COLUMN_NAMES)
input_df.show()

SnowparkML also includes metric calculations such as correlations, and more. We will use the SnowparkML correlation method on our input dataframe to identify any linearly correlated features to the output. We'll also use matplotlib to plot the resulting matrix. Notice that all of the correlation calculations are pushed down to Snowflake!



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from snowflake.ml.modeling.metrics.correlation import correlation
corr_df = correlation(df=input_df)

In [ ]:
# Return only the correlation summary results to generate this heatmap
sns.heatmap(corr_df, cmap="YlGnBu", annot=True)

What's great about this, is that we are using a lot of Snowpark components under the hood- the dataframe API, SQL, Python stored procedures and more. But with the new SnowparkML API, data scientists can take advantage of all that Snowpark affords them, while using common, familiar APIs that match how they do their work today.

Now that we have our feature data, let's actually fit an XGBoost model to our features to attempt to predict future sales. We'll fit several different models with different hyperparameters, and then show how we can use the Snowpark Model Registry to select our best-fit model.



In [ ]:
# Split the data into train and test sets
train_df, test_df = input_df.random_split(weights=[0.9, 0.1], seed=98)

## What's happening when you leverage Snowpark ML Modeling API?

Let's run our training job using the SnowparkML Modeling API- this will push down our model training to run on Snowflake, and you'll notice that the type of the model object returend is a SnowparkML XGBClassifier- this has some benefits, but also is fully compatible with the standard sklearn/xgboost model objects.

- The model.fit() function actually creates a temporary stored procedure in the background. This also means that the model training is a single-node operation. Be sure to use a Snowpark Optimized Warehouse if you need more memory. We are just using an XS Standard Virtual Warehouse here, which we created at the beginning of this quickstart.
- The model.predict() function actually creates a temporary vectorized UDF in the background, which means the input DataFrame is batched as Pandas DataFrames and inference is parallelized across the batches of data. You can check the query history once you execute the following cell to check.

<img src="assets/Snowpark_ML_API.png" width="800" />

In [ ]:
from snowflake.ml.modeling.xgboost import XGBRegressor
# Define the XGBRegressor
regressor = XGBRegressor(
    label_cols = LABEL_COLUMNS,
    input_cols = FEATURE_COLUMN_NAMES,
    output_cols = OUTPUT_COLUMNS
)

# Train
regressor.fit(train_df)

# Predict
result = regressor.predict(test_df)

In [ ]:
# Just to illustrate, we can also pass in a Pandas DataFrame to Snowpark ML's model.predict()
regressor.predict(test_df.to_pandas())
# try 
regressor.predict(test_df)

Let's analyze the results using Snowpark ML's MAPE.



In [ ]:
# Use Snowpark ML metrics to calculate
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error, mean_squared_error

# Predict
results = regressor.predict(test_df)

# Calculate MAPE
mape = mean_absolute_percentage_error(df=results, y_true_col_names=LABEL_COLUMNS, y_pred_col_names=OUTPUT_COLUMNS)

# Calculate MSE
mse = mean_squared_error(df=results, y_true_col_names=LABEL_COLUMNS, y_pred_col_names=OUTPUT_COLUMNS)

results.select([*LABEL_COLUMNS, *OUTPUT_COLUMNS]).show()
print(f'''Mean absolute percentage error: {mape}''')
print(f'''Mean squared error: {mse}''')

In [ ]:
# Plot actual vs predicted 
g = sns.relplot(data=results["SHIFT_SALES", "PRED_SHIFT_SALES"].to_pandas().astype("float64"), x="SHIFT_SALES", y="PRED_SHIFT_SALES", kind="scatter")
g.ax.axline((0,0), slope=1, color="r")

plt.show()

## Snowpark ML's GridSearchCV()

Now, let's use Snowpark ML's GridSearchCV() function to find optimal model parameters.



In [ ]:
from snowflake.ml.modeling.model_selection import GridSearchCV

grid_search = GridSearchCV(
estimator=XGBRegressor(),
param_grid={
"n_estimators":[25, 50],
"learning_rate":[0.4, 0.5],
},
n_jobs = -1,
scoring="neg_mean_absolute_percentage_error",
input_cols=FEATURE_COLUMN_NAMES,
label_cols=LABEL_COLUMNS,
output_cols=OUTPUT_COLUMNS
)


# Train
grid_search.fit(train_df)

In [ ]:
# Let's analyze the grid_search results
gs_results = grid_search.to_sklearn().cv_results_
n_estimators_val = []
learning_rate_val = []
for param_dict in gs_results["params"]:
    n_estimators_val.append(param_dict["n_estimators"])
    learning_rate_val.append(param_dict["learning_rate"])
mape_val = gs_results["mean_test_score"]*-1

gs_results_df = pd.DataFrame(data={
    "n_estimators":n_estimators_val,
    "learning_rate":learning_rate_val,
    "mape":mape_val})

sns.relplot(data=gs_results_df, x="learning_rate", y="mape", hue="n_estimators", kind="line")

plt.show()

In [ ]:
# Let's save our optimal model and its metadata:
optimal_model = grid_search.to_sklearn().best_estimator_
optimal_n_estimators = grid_search.to_sklearn().best_estimator_.n_estimators
optimal_learning_rate = grid_search.to_sklearn().best_estimator_.learning_rate


optimal_mape = gs_results_df.loc[(gs_results_df['n_estimators']==optimal_n_estimators) &
                        (gs_results_df['learning_rate']==optimal_learning_rate),'mape'].values[0]

## **Scale down your assigned Snowflake compute warehouse.**



In [ ]:
# Decrease size of Snowflake compute warehouse to XSMALL
session.sql("alter warehouse " + warehouse + " set warehouse_size = XSMALL").collect()

# Part 4 - Use Snowpark for MLOps

<img src="assets/Part4.png" width="800" />

<img src="assets/model_deployment.png" width="800" />

## Now let's use Snowflake's ML Model Registry

Model Registry was created to support model management operations including model registration, versioning, metadata and audit trails. Integrated deployment infrastructure for batch inference is a critical ease-of-use feature. Users can deploy ML models for batch inference from the registry directly into a Snowflake Warehouse as a vectorized UDF, or as a service to a customer-specified Compute Pool in Snowpark Container Services.

Snowflake's Model Registry supports SciKitLearn, XGBoost, Pytorch, Tensorflow and MLFlow (via the pyfunc interface) models.

<img src="assets/ModelRegistry.png" width="800" />


Model Registry allows easy deployment of pre-trained open-source models from providers such as HuggingFace. See this blog for more details: [https://medium.com/snowflake/whats-in-a-name-model-naming-versioning-in-snowpark-model-registry-b5f7105fd6f6](https://medium.com/snowflake/whats-in-a-name-model-naming-versioning-in-snowpark-model-registry-b5f7105fd6f6) or the Model Registry documentation: [https://docs.snowflake.com/LIMITEDACCESS/snowflake-ml-model-registry](https://docs.snowflake.com/LIMITEDACCESS/snowflake-ml-model-registry)



## Create the Model Registry and register your model



In [ ]:
from snowflake.ml.registry import Registry

native_registry = Registry(session, database_name="HOL", schema_name=schema)

In [ ]:
# Get sample input data to pass into the registry logging function
X = train_df.select(FEATURE_COLUMN_NAMES).limit(100)

# Define model name
model_name = "SHIFT_SALES_PREDICTION"

# Let's first log the very first model we trained
model_ver = native_registry.log_model(
    model_name=model_name,
    version_name='V0',
    model=regressor,
    sample_input_data=X, # to provide the feature schema
)

In [ ]:
# Now, let's log the optimal model from GridSearchCV
model_ver2 = native_registry.log_model(
    model_name=model_name,
    version_name='V2',
    model=optimal_model,
    sample_input_data=X, # to provide the feature schema
)

# Add evaluation metric
model_ver2.set_metric(metric_name="mean_abs_pct_err", value=optimal_mape)

# Add a description
model_ver2.comment = "This is the second iteration of the Shift Sales Price Prediction model \
                        where we performed hyperparameter optimization."

In [ ]:
# Let's confirm model(s) that were added
native_registry.get_model(model_name).show_versions()

In [ ]:
# We can see what the default model is when we have multiple versions with the same model name:
native_registry.get_model(model_name).default.version_name

In [ ]:
# Now we can use the default version model to perform inference.
model_ver = native_registry.get_model(model_name).version('V0')
result_sdf = model_ver.run(test_df, function_name="predict")
result_sdf.show()

<img src="assets/model_utilization.png" width="800" />

Now that our model is built and deployed, let's see it in action! We will find the best place to park in Vancouver for tomorrow morning's shift.



In [ ]:
date_tomorrow_df = oe_df.filter(
    (F.col("shift_sales").isNull())
    & (F.col("shift_oe") == 1)
    & (F.col("city") == "Vancouver")
)
date_tomorrow_df.show()

In [ ]:
result_sdf = regressor.predict(date_tomorrow_df)
result_sdf.show()

## Visualize on a Map

The red and yellow areas indicate higher predicted sales locations and the green zones indicate lower predicted sales. We will use this insight to ensure that our drivers are parking at the high-value locations. Value: Updated predictions readily available to drive towards our corporate goals.



# Part 5 - Create a SiS application to use predicted outputs

<img src="assets/Part5.png" width="800" />


Create a SiS application for local managers to identify where to place daily food trucks. 



### Create new SiS app. 

### Replace existing code with SiS_application.py code listed below.

### Update database in lines 41, 66, and 124. 

### Include required pydeck and snowflake-ml-python package. 

### Run the SiS application.

<img src="assets/SiSapp.png" width="800" />

```
# Import Python packages

import streamlit as st

import pydeck as pdk

import numpy as np





# Import Snowflake modules

from snowflake.snowpark import Session

import snowflake.snowpark.functions as F

from snowflake.snowpark import Window

from snowflake.snowpark.context import get_active_session

from snowflake.ml.registry import Registry

import snowflake.ml.modeling.preprocessing as snowmlpp





# Set Streamlit page config

st.set_page_config(

    page_title="Streamlit App: Snowpark 101", 

    page_icon=":truck:",

    layout="wide",

)





# Add header and a subheader

st.header("Predicted Shift Sales by Location")

st.subheader("Data-driven recommendations for food truck drivers.")

 





# Connect to Snowflake

# session = init_connection()

session = get_active_session()

 

# Create input widgets for cities and shift

with st.container():

    col1, col2 = st.columns(2)

    with col1:

        # Drop down to select city

        city = st.selectbox(

            "City:",

            session.table("HOL.SCHEMA0.SHIFT_SALES_V")

            .select("city")

            .distinct()

            .sort("city"),

        )

 

    with col2:

        # Select AM/PM Shift

        shift = st.radio("Shift:", ("AM", "PM"), horizontal=True)





    n_trucks = st.selectbox('How many food trucks would you like to schedule today?', np.arange(1,10))





    if n_trucks > 1:

        range = st.slider('What is the minimum distance in kilometers between food trucks?', 0, 20, 1)

        st.write('You are requesting a minimum distance of ', range, 'km')

        st.write('Click **:blue[Update]** to get the ', n_trucks, ' highest predicted Shift_Sales food truck locations.')

    else:

        st.write('Click **:blue[Update]** to get one food truck location predicted to have the Shift_Sales')

        

# Get predictions for city and shift time

def get_predictions(city, shift):

    # Get data and filter by city and shift

    snowpark_df = session.table(

        "HOL.SCHEMA0.SHIFT_SALES_V"

    ).filter((F.col("shift") == shift) & (F.col("city") == city))

 

    # Get rolling average

    window_by_location_all_days = (

        Window.partition_by("location_id")

        .order_by("date")

        .rows_between(Window.UNBOUNDED_PRECEDING, Window.CURRENT_ROW - 1)

    )

 

    snowpark_df = snowpark_df.with_column(

        "avg_location_shift_sales",

        F.avg("shift_sales").over(window_by_location_all_days),

    ).cache_result()

 

    # Get tomorrow's date

    date_tomorrow = (

        snowpark_df.filter(F.col("shift_sales").is_null())

        .select(F.min("date"))

        .collect()[0][0]

    )

 

    # Filter to tomorrow's date

    snowpark_df = snowpark_df.filter(F.col("date") == date_tomorrow)

 

    # Impute

    snowpark_df = snowpark_df.fillna(value=0, subset=["avg_location_shift_sales"])





    for colname in snowpark_df.columns:

        new_colname = str.upper(colname)

        snowpark_df = snowpark_df.with_column_renamed(colname, new_colname)

 

    # Encode

    snowpark_df = snowpark_df.with_column("shift_oe", F.iff(F.col("shift") == "AM", 0, 1))\

                             .with_column("shift_oe", F.iff(F.col("shift") == "PM", 1, 0))



    # Scale

    mm_target_columns = ["CITY_POPULATION"]

    mm_target_cols_out = ["CITY_POPULATION_NORM"]

    snowml_mms = snowmlpp.MinMaxScaler(input_cols=mm_target_columns, 

                                       output_cols=mm_target_cols_out)

    snowml_mms.fit(snowpark_df)

    snowpark_df = snowml_mms.transform(snowpark_df)

    

    # Get all features

    feature_cols = ["SHIFT_OE", 

                    "CITY_POPULATION_NORM", 

                    "MONTH", 

                    "DAY_OF_WEEK",

                    "LATITUDE",

                    "LONGITUDE",

                    "AVG_LOCATION_SHIFT_SALES",

                    "LOCATION_ID"]





    snowpark_df = snowpark_df.select(feature_cols)



    native_registry = Registry(session=session, database_name="HOL", schema_name="SCHEMA0")

    model_ver = native_registry.get_model("SHIFT_SALES_PREDICTION").version('v0')

    result_sdf = model_ver.run(snowpark_df, function_name="predict")

    return result_sdf



# Update predictions and plot when the "Update" button is clicked

if st.button(":blue[Update]"):

    # Get predictions

    with st.spinner("Getting predictions..."):

        predictions_sdf = get_predictions(city, shift)

        predictions = predictions_sdf.to_pandas()

 

    # Plot on a map

    st.subheader("Predicted Shift Sales for position")

    predictions["PRED_SHIFT_SALES"].clip(0, inplace=True)

    st.pydeck_chart(

        pdk.Deck(

            map_style=None,

            initial_view_state=pdk.ViewState(

                latitude=predictions["LATITUDE"][0],

                longitude=predictions["LONGITUDE"][0],

                zoom=11,

                pitch=50,

            ),

            layers=[

                pdk.Layer(

                    "HexagonLayer",

                    data=predictions,

                    get_position="[LONGITUDE, LATITUDE]",

                    radius=200,

                    elevation_scale=4,

                    elevation_range=[0, 1000],

                    pickable=True,

                    extruded=True,

                ),

                pdk.Layer(

                    "ScatterplotLayer",

                    data=predictions,

                    get_position="[LONGITUDE, LATITUDE]",

                    get_color="[200, 30, 0, 160]",

                    get_radius=200,

                ),

            ],

        )

    )

    

    max_x = predictions.loc[predictions["PRED_SHIFT_SALES"].idxmax()]

    st.write("Maximum Predicted Sales are expected at the following location:", max_x)

    #st.dataframe(predictions_sdf)

    

    location_id = max_x["LOCATION_ID"]

    lat = max_x["LATITUDE"]

    long = max_x["LONGITUDE"]



    st.subheader("The following chart is generated using the st_point and st_distance Snowflake Geospatial features")



    if n_trucks == 1:

        st.write("Have your only food truck positioned at Location ID ", location_id, " to maximize SHIFT_SALES")

    elif n_trucks > 1:

        best_locations = [location_id]

        available_locations_sdf = predictions_sdf

    

        st_distance = F.function('st_distance')

        st_point = F.function('st_point')

    

        for truck_n in np.arange(0,n_trucks - 1):

            available_locations_sdf = available_locations_sdf.with_column("DISTANCE_TO_TRUCK", 

                                        st_distance(

                                            st_point(F.lit(float(long)), F.lit(float(lat))),

                                            st_point(F.col("LONGITUDE"), F.col("LATITUDE"))

                                        )/1609

                                       ).filter(F.col("DISTANCE_TO_TRUCK") >= range/1.609).order_by("PRED_SHIFT_SALES", ascending=False)

            max_x = available_locations_sdf.limit(1).to_pandas()

            try:

                location_id = max_x["LOCATION_ID"].iloc[0]

                lat = max_x["LATITUDE"].iloc[0]

                long = max_x["LONGITUDE"].iloc[0]

            except:

                break

            best_locations.append(location_id)





        selected_locations = predictions[predictions["LOCATION_ID"].isin(best_locations)]

        st.map(selected_locations)

        st.dataframe(selected_locations)
```
